## <p style="text-align: right;"> &#9989; Nic Wiggins </p>

In order to successfully complete this assignment, you must follow all instructions in this notebook and upload your edited ipynb file with your answers on or before **11:59pm on Friday Feb. 8th**.

**Please read though the entire project before starting!**

This homework uses "the language of linear algebra" to describe how to solve a common engineering problem efficiently. 
You may not be familiar with some of the terms. 
Try to see if you can figure them out in context or use your favorite search engine. 
If you still don't understand, stop by office hours.


# Homework 2: Gaussian Elimination, LU Decomposition, and PDEs


<img src="https://exclaim.ca/images/Marc_Maron_Thinky_Pain.jpg">

In this homework, you will create algorithms that factorize invertible matrices (i.e., square matrices with linearly independent columns), $A$, into the following decompositions (**note**: the terms *decomposition* and *factorization* are used interchangeably in literature):

- LU Decomposition: $A = LU$
- Cholesky Decomposition: $A = R^TR\quad (= LDL^T)$
    - This factorization of $A$ into $R^TR$ also requires that $A$ be *symmetric* and *positive-definite*. The latter term simply requires that $x^{T}Ax > 0$ for every nonzero $x \in \mathbb{R}^n$. If it is not obvious, you should confirm that $x^{T}Ax$ is always some scalar value (e.g., note that $x^TA = y^T$ for some vector $y\in\mathbb{R}^n$, and $y^Tx$ is the dot product between $x$ and $y$ and, hence, a real scalar).
    
Each matrix in these decompositions is the *matrix product* of <a href="https://en.wikipedia.org/wiki/Elementary_matrix">elementary matrices</a>. 
Recall that *an elementary matrix differs from the identity matrix (the square matrix with $1$s on the diagonal and $0$s elsewhere) by an elementary row operation*.
 
The use of these matrix decompositions in Numerical Linear Algebra is motivated by computational efficiency or reduction of *computational complexity* (recall "**Big-O notation**" and **counting the loops in your matrix multiplication algorithm**) and *numerical stability*. 
Solving our old friend $Ax = b$ by computing the inverse of $A$, denoted $A^{-1}$, and taking the matrix-vector product $A^{-1}b = x$ is computational resource intensive and numerically unstable, in general. 
If the $LU$ decomposition of $A$ exists, then it will be less costly and more stable to:
1. Solve $Ly = b$ for $y$ by *forward-substitution*; and then
2. Solve $Ux = y$ for $x$ by *backward-substitution*

A final note to relate Homework 2 to Homework 1: The algorithms presented here are of a different class than those presented in the first homework. 
The **Jacobi Algorithm** and **Gauss-Siedel Algorithm** are *iterative algorithms*. 
As you now know, this means that the algorithmic procedure is applied once, twice, and so on, *ad infinitum*, until the output is within a desired tolerance, or until the process has been executed a given number of times (e.g., 1000 iterations). 
To contrast, the algorithms in this homework are executed in *finite time*. 
You have already written such an algorithm: **Matrix Multiply** and **Gauss-Jordan Elimination**. 
It may be useful to bear this distinction in mind as you encounter new algorithms.  

----
# Part 0: Motivations

Imagine we have a metal rod that is 1 meter in length and 1 centimeter in diameter. 
Suppose that we keep each end cooled to $0^{\circ}$ Celsius and heat the middle to $100^{\circ}$ Celsius. 
Then, we let the rod cool and record the temperature distribution across its length.

<img src="https://tgneast.blob.core.windows.net/content/wp-content/uploads/2014/11/sh74.jpg" width='300'>

We will model this phenomenon using the 1-Dimensional Heat Equation, a Partial Differential Equation (PDE) which relates the first derivative in time of a function, $u(x,t)$, to its second derivative in space. This is formally stated as:

$$\frac{\partial u(x,t)}{\partial t} = \frac{\partial^{2} u(x,t)}{\partial x^{2}}.$$

In order for our problem to be well-posed, we will use the following boundary and initial conditions:

$$u(0,t) = u(1,t) = 0, x \in [0,1] $$
$$u(x,0) = 100\sin(\pi x) $$ 

While not obvious from the equation above, when this PDE is solved numerically (e.g., by a computer), the problem reduces to one of Linear Algebra, namely $Ax=b$. 
To solve this equation, we need to invert the matrix $A$ or find a clever way to circumvent doing so. 
Let's attempt the latter.

**Note**: This assignment covers a diverse range of concepts. 
You are only responsible for Linear Algebra content.

----
# Part 1: Gaussian Elimination & LU Decomposition

Recall that Gaussian Elimination converts an arbitrary matrix, $A$, into its *row echelon form*. 
For our purposes, let's suppose that $A$ is a square $n\times n$ matrix. 
To simplify our tasks, let us further impose the condition that $A$ is invertible. 
Thus, the columns of $A$ are linearly independent. 
This means that Gaussian Elimination will yield an ***upper-triangular*** *matrix*. 
Let us denote this matrix $U$ for ***upper-triangular***.

If there were a function, $f$ that could take $A$ and output $U$, we could think of Gaussian Elimination as the following process:

$$f(A)=U$$

In fact, we have such a function, or linear map, $L^{-1}$. 
$L^{-1}$ is a ***unit lower-triangular*** *matrix*. 
This means that $L^{-1}$ has $1$s on its diagonal and $0$s above. 
Elements below the diagonal are real numbers.

$$L^{-1} = 
\left(
    \begin{array}{*5{c}}
         1 &  0 &  0 &  0 &  0 \\
         -l_{21} &  1 &  0 &  0 &  0 \\
         -l_{31} &  -l_{32} &  1 &  0 &  0 \\
         -l_{41} &  -l_{42} &  -l_{43} &  1 &  0 \\
         -l_{51} &  -l_{52} &  -l_{53} &  -l_{54} &  1 \\
      \end{array}
  \right)
$$

(Fret not, dear reader: The use of $-l_{ij}$ instead of $l_{ij}$ for the elements of $L^{-1}$ will become clear in a moment. However, note that for any unit lower-triangular matrix, its subdiagonal entries, $l_{ij}, i>j$, may take any value, so, without loss of generality, we may choose $-l_{ij}$ for its entries.)

With this information, we may now rewrite our equation from above as:

$$L^{-1}A = U$$

You may have noticed the superscript in $L^{-1}$. 
This just says that $L^{-1}$ is the inverse of some matrix $L$. 
And for any invertible matrix, $L$, we have that the matrix products:

$$L^{-1}L = LL^{-1} = I$$

This is analogous to (for every real number $a\neq 0$): 

$$a^{-1}\times a = a\times a^{-1} = 1$$

To understand $L^{-1}$ more deeply, let's turn our attention back to Gaussian Elimination for a moment. For a "sufficiently nice" $n\times n$ matrix $A$, the matrix $L^{-1}$ that takes $A$ to its ***upper-triangular*** or ***row echelon form***, $U$, has the structure:

$$L^{-1} = L_{n-1}L_{n-2}...L_{2}L_{1}$$

Each of the $L_{i}$s above is an elementary matrix that zeros out the subdiagonal entries of the $i^{th}$ column of $A$. This is **the $i^{th}$ step of** ***Gaussian Elimination*** **applied to** ***the entire $i^{th}$ column of A below the $i^{th}$ diagonal element***.  

Let's show this by computation of $L_i$ for a "nice" matrix $A$.

In [1]:
## Import all necessary packages
%matplotlib inline
import scipy.sparse as sparse #this helps to speed up the algorithms, but you will not use it. 
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
sym.init_printing(use_unicode=True)

## These will allow us to see a cool simulation of the Heat Equation problem (if we compute our answers correctly!)
from matplotlib import animation, rc
from IPython.display import HTML

In [2]:
L = np.matrix([[1 , 0, 0], [-2, 1,0], [4,2,1]])
sym.Matrix(L)

⎡1   0  0⎤
⎢        ⎥
⎢-2  1  0⎥
⎢        ⎥
⎣4   2  1⎦

In [3]:
sym.Matrix(np.linalg.inv(L))

⎡1.0   0.0   0.0⎤
⎢               ⎥
⎢2.0   1.0   0.0⎥
⎢               ⎥
⎣-8.0  -2.0  1.0⎦

## Gaussian Elimination by Elementary Matrices, $L_i$

Let $A$ be the following matrix:

$$A = 
\begin{bmatrix}
    2 &  1 &  1 &  0 \\
    4 &  3 &  2 &  1 \\
    8 &  7 &  8 &  5 \\
    6 &  7 &  9 &  8 \\
\end{bmatrix}
$$

<font color=red>**DO THIS:**</font> Create a $4 \times 4$ **unit lower-triangular** matrix, $L_1$ that eliminates all of the subdiagonal entries of $A$ in the first column. Display the matrix $L_1$ using SymPy.

In [4]:
A = np.matrix([[2,1,1,0],[4,3,2,1],[8,7,8,5],[6,7,8,8]]) # Here is A for your convenience
As = sym.Matrix(A)
As

⎡2  1  1  0⎤
⎢          ⎥
⎢4  3  2  1⎥
⎢          ⎥
⎢8  7  8  5⎥
⎢          ⎥
⎣6  7  8  8⎦

In [5]:
## Type your answer here ##
L1 = np.matrix([[1,0,0,0],[-2,1,0,0],[-4,0,1,0],[-3,0,0,1]])
sym.Matrix(L1)

⎡1   0  0  0⎤
⎢           ⎥
⎢-2  1  0  0⎥
⎢           ⎥
⎢-4  0  1  0⎥
⎢           ⎥
⎣-3  0  0  1⎦

In [6]:
L1*A

matrix([[2, 1, 1, 0],
        [0, 1, 0, 1],
        [0, 3, 4, 5],
        [0, 4, 5, 8]])


We should now have the following:

$$L_{1}A = A^{(1)} =
\begin{bmatrix}
    2 &  1 &  1 &  0 \\
    0 &  u_{22} &  u_{23} &  u_{24} \\
    0 &  x &  x &  x \\
    0 &  x &  x &  x \\
\end{bmatrix}
=
\begin{bmatrix}
    u_{11} &  u_{12} &  u_{13} &  u_{14} \\
    0 &  u_{22} &  u_{23} &  u_{24} \\
    0 &  x &  x &  x \\
    0 &  x &  x &  x \\
\end{bmatrix}
$$

Since our first row remained unchanged, we know that our $u_{1i}$ (the first row entries of $U$) are now determined. Similarly, we have that the $u_{2i}$ (the second row entries of $U$) are determined as well. The $x$ elements are elements that have changed, but are not yet in their final form.  **Note**: Your $u_{ij}$ will be whole, or Integer ($\mathbb{Z}$), numbers. 

 <font color=red>**DO THIS:**</font> Left-multiply $A$ by $L_1$ to confirm that all of the subdiagonal entries of the first column of $A^{(1)}$ are zero. Display the result via SymPy.

In [7]:
## Type your answer here ##
A1 = L1*A
sym.Matrix(A1)

⎡2  1  1  0⎤
⎢          ⎥
⎢0  1  0  1⎥
⎢          ⎥
⎢0  3  4  5⎥
⎢          ⎥
⎣0  4  5  8⎦

Our next step will be to eliminate all nonzero entries from the second column of $A^{(1)} = L_{1}A$ by left multiplication of $L_{2}$. This should yield: 

$$\begin{align}A^{(2)} &= L_{2}A^{(1)} \\
&= L_{2}L_{1}A \\
&=
\begin{bmatrix}
    u_{11} &  u_{12} &  u_{13} &  u_{14} \\
    0 &  u_{22} &  u_{23} &  u_{24} \\
    0 &  0 &  u_{33} &  u_{34} \\
    0 &  0 &  x &  x \\
\end{bmatrix}
\end{align}
$$

<font color=red>**DO THIS:**</font> Create a $4 \times 4$ **unit lower-triangular** matrix, $L_2$ that eliminates all of the subdiagonal entries of the second column of $A^{(1)}$ yielding $A^{(2)}$ as above. Display the matrix $L_2$ using SymPy.

In [8]:
## Type your answer here ##
L2 = np.matrix([[1,0,0,0],[0,1,0,0],[0,-3,1,0],[0,-4,0,1]]) # for your convenience
sym.Matrix(L2)

⎡1  0   0  0⎤
⎢           ⎥
⎢0  1   0  0⎥
⎢           ⎥
⎢0  -3  1  0⎥
⎢           ⎥
⎣0  -4  0  1⎦

 <font color=red>**DO THIS:**</font> Left-multiply $A^{(1)}$ by $L_2$ to confirm that all of the subdiagonal entries of column 2 of $A^{(2)}$ are zero. Display the result via SymPy. **Note**: Your $u_{ij}$ will be whole, or Integer ($\mathbb{Z}$), numbers. 

In [9]:
## Type your answer here ##
A2 = L2*A1
sym.Matrix(A2)

⎡2  1  1  0⎤
⎢          ⎥
⎢0  1  0  1⎥
⎢          ⎥
⎢0  0  4  2⎥
⎢          ⎥
⎣0  0  5  4⎦

We should now have:

$$
\begin{align}A^{(2)} &= L_{2}A^{(1)} \\
&= L_{2}L_{1}A \\
&=
\begin{bmatrix}
    u_{11} &  u_{12} &  u_{13} &  u_{14} \\
    0 &  u_{22} &  u_{23} &  u_{24} \\
    0 &  0 &  u_{33} &  u_{34} \\
    0 &  0 &  x &  x \\
\end{bmatrix}
\end{align}
$$

We now want to build the final matrix $L_{3}$ that will take our matrix $A^{(2)}$ to ***upper-triangular*** **form**. So, let's do that!

 <font color=red>**DO THIS:**</font> Create a $4 \times 4$ **unit lower-triangular** matrix, $L_3$ that eliminates all of the subdiagonal entries of the third column of $A^{(2)}$ yielding: 

$$
\begin{align}A^{(3)} &= L_{3}A^{(2)} \\ 
&= L_{3}L_{2}A^{(1)} \\
&= L_{3}L_{2}L_{1}A \\
&= U
\end{align}
$$. 

Display the matrix $L_3$ using SymPy.

In [10]:
## Type your answer here ##
L3 = np.matrix([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,-5/4,1]]) # for your convenience
sym.Matrix(L3)

⎡1.0  0.0   0.0   0.0⎤
⎢                    ⎥
⎢0.0  1.0   0.0   0.0⎥
⎢                    ⎥
⎢0.0  0.0   1.0   0.0⎥
⎢                    ⎥
⎣0.0  0.0  -1.25  1.0⎦

<font color=red>**DO THIS:**</font> Left-multiply $A^{(2)}$ by $L_3$ to confirm that all of the subdiagonal entries of column 3 of $A^{(3)}$ are zero. Display the result via SymPy. **Note**: Your $u_{ij}$ will be whole, or Integer ($\mathbb{Z}$), numbers. You should now notice that $A^{(3)} = U$ is in **row echelon form**, and, hence, $U$ is an **upper-triangular matrix** with $0$s below the diagonal!

In [11]:
## Type your answer here ##
A3 = L3*A2
sym.Matrix(A3)

⎡2.0  1.0  1.0  0.0⎤
⎢                  ⎥
⎢0.0  1.0  0.0  1.0⎥
⎢                  ⎥
⎢0.0  0.0  4.0  2.0⎥
⎢                  ⎥
⎣0.0  0.0  0.0  1.5⎦

In [12]:
sym.Matrix(L3*L2*L1*A)

⎡2.0  1.0  1.0  0.0⎤
⎢                  ⎥
⎢0.0  1.0  0.0  1.0⎥
⎢                  ⎥
⎢0.0  0.0  4.0  2.0⎥
⎢                  ⎥
⎣0.0  0.0  0.0  1.5⎦

In [13]:
L3*L2*L1

matrix([[ 1.  ,  0.  ,  0.  ,  0.  ],
        [-2.  ,  1.  ,  0.  ,  0.  ],
        [ 2.  , -3.  ,  1.  ,  0.  ],
        [ 2.5 , -0.25, -1.25,  1.  ]])

In [14]:
sym.Matrix(np.linalg.inv(L3*L2*L1))

⎡1.0  1.99840144432528e-16  1.58603289232165e-17  0.0⎤
⎢                                                    ⎥
⎢2.0          1.0           1.58603289232165e-16  0.0⎥
⎢                                                    ⎥
⎢4.0          3.0                   1.0           0.0⎥
⎢                                                    ⎥
⎣3.0          4.0                   1.25          1.0⎦

### Congratulations! 

You have just decomposed your first matrix via the process below (and should now have a matrix, $U$, that looks like the one below):

$$
\begin{align}L^{-1}A &= L_{3}L_{2}L_{1}A \\
&= L_{3}L_{2}A^{(1)} \\
&= L_{3}A^{(2)} \\
&= A^{(3)} \\
&= U \\
&=
\begin{bmatrix}
    2 & 1 & 1 & 0 \\
    0 & 1 & 0 & 1 \\
    0 & 0 & 4 & 2 \\
    0 & 0 & 0 & 1.5
\end{bmatrix}
\end{align}
$$

 <font color=red>**DO THIS:**</font>

Finally, let's explicitly generate the matrices $L^{-1}$ and $L$. Then, display them using SymPy.

It will be helpful to use the following:

$$\begin{align}L^{-1} &= L_{n-1}L_{n-2}...L_{2}L_{1}\end{align}$$
and
$$\begin{align}L &= (L^{-1})^{-1} \\
&= (L_{n-1}L_{n-2}...L_{2}L_{1})^{-1} \\
&= L_{1}^{-1}L_{2}^{-1}...L_{n-2}^{-1}L_{n-1}^{-1}
\end{align}
$$



In [15]:
## Type your answer here ##
Linv = L3*L2*L1
L = np.linalg.inv(L1)*np.linalg.inv(L2)*np.linalg.inv(L3)
sym.Matrix(Linv)

⎡1.0    0.0    0.0   0.0⎤
⎢                       ⎥
⎢-2.0   1.0    0.0   0.0⎥
⎢                       ⎥
⎢2.0   -3.0    1.0   0.0⎥
⎢                       ⎥
⎣2.5   -0.25  -1.25  1.0⎦

In [16]:
sym.Matrix(L3*L2*L1)

⎡1.0    0.0    0.0   0.0⎤
⎢                       ⎥
⎢-2.0   1.0    0.0   0.0⎥
⎢                       ⎥
⎢2.0   -3.0    1.0   0.0⎥
⎢                       ⎥
⎣2.5   -0.25  -1.25  1.0⎦

In [17]:
sym.Matrix(L)

⎡1.0  0.0  0.0   0.0⎤
⎢                   ⎥
⎢2.0  1.0  0.0   0.0⎥
⎢                   ⎥
⎢4.0  3.0  1.0   0.0⎥
⎢                   ⎥
⎣3.0  4.0  1.25  1.0⎦

For our last bit of LU Decomposition fun, let's confirm that your matrices $L$ and $U$ fulfill the equality:

$$A = LU$$

Indeed, there is a function in SymPy that will compute the LU decomposition for us.

 <font color=red>**DO THIS:**</font> Run the following function and print its outputs: 

$$\texttt{L_actual, U_actual, _ = As.LUdecomposition()}$$

Then, compute:

$$\texttt{L_actual*U_actual - As}$$

and confirm that it outputs the zero matrix.

In [18]:
## Type your answer here ##
L_actual, U_actual, _ = As.LUdecomposition()
print(L_actual)
print(U_actual)

L_actual*U_actual - As

Matrix([[1, 0, 0, 0], [2, 1, 0, 0], [4, 3, 1, 0], [3, 4, 5/4, 1]])
Matrix([[2, 1, 1, 0], [0, 1, 0, 1], [0, 0, 4, 2], [0, 0, 0, 3/2]])


⎡0  0  0  0⎤
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎣0  0  0  0⎦

----
# Part 2: Numerical Solution to the Heat Equation (PDE)

Let us return to the Heat Equation presented in Part 0.


## Symmetric Matrices

From discretization of partial differential equations (PDEs) to matrix representations of networks to covariance matrices in statistics, symmetric matrices often arise through applications of Linear Algebra. A ***symmetric matrix*** is a matrix, $A$, for which $A = A^T$ (i.e., for every entry $(A)_{ij}$ {also written $a_{ij}$} of $A$ we have that $(A)_{ij} = (A)_{ji}$). An immediate consequence of this is that $A$ must be square. 

Symmetric matrices for which $x^TAx > 0$ for all nonzero $x\in \mathbb{R}^n$ are called ***positive-definite (PD)***. 
Symmetric positive-definite matrices are desirable from a computational standpoint, as they admit efficient solutions to $Ax=b$ via the Cholesky decomposition. 
Cholesky decomposition is very closely related to the LU decomposition, so we will build an algorithm to generate $LU$ from a matrix $A$ and then use the decomposition to numerically solve the heat equation.

### General LU Decomposition Algorithm 

 <font color=red>**DO THIS:**</font>  Using the scaffolded code below, complete the LU decomposition algorithm. (It may be helpful to test your code on the matrix $A$ from above.)

In [19]:
## Type your answer here ##
## You just NEED to fill in the missing places in the codes ##
A = np.matrix([[2,1,1,0],[4,3,3,1],[8,7,9,5],[6,7,9,8]]) # to test

def LU_decomp(B):
    n = len(B)
    U = B.copy() + 0.0 # make sure subtraction with floats resuts in floats - was getting int subtraction
    L = np.identity(n)
    for k in np.arange(0,n-1):
        for j in np.arange(k+1,n):
            L[j,k] = U[j,k]/U[k,k]
            U[j,k:n] = U[j,k:n] - L[j,k]*U[k,k:n]
    return L, U

L1,U1 = LU_decomp(A) # syntax for returning matrices
print(L1)
print(U1)
np.linalg.norm(L1*U1 - A) # Test: should return 0

[[1. 0. 0. 0.]
 [2. 1. 0. 0.]
 [4. 3. 1. 0.]
 [3. 4. 1. 1.]]
[[2. 1. 1. 0.]
 [0. 1. 1. 1.]
 [0. 0. 2. 2.]
 [0. 0. 0. 2.]]


In [20]:
A = np.matrix([[6,2,-2],[1,4,1],[2,2,5]])
b = np.matrix([[-2],[4],[32]])

L1,U1 = LU_decomp(A) # syntax for returning matrices
print(L1)
print(U1)
np.linalg.norm(L1*U1 - A) # Test: should return 0


[[1.         0.         0.        ]
 [0.16666667 1.         0.        ]
 [0.33333333 0.36363636 1.        ]]
[[ 6.          2.         -2.        ]
 [ 0.          3.66666667  1.33333333]
 [ 0.          0.          5.18181818]]


### Solve $Ax=b$ via LU Decomposition

You may wish to refer to the introduction of this assignment for a general overview of how to use LU decomposition to solve $Ax = b$.

 <font color=red>**DO THIS:**</font>  Using the scaffolded code below, complete the LU solver algorithm. 
The algorithm should solve $Ly = b$ for $y$ via Forward-Substitution and then $Ux=y$ for $x$ by Backward-Substitution. 
(It may be helpful to test your code on a matrix $A$ and vector $b$ from homework 1 or another source.)

In [21]:
## Type your answer here ##
## You just NEED to fill in the missing places in the codes ##
def LU_Axb_solve(A,b):
    L,U = LU_decomp(A)
    n = len(A)
    # Forward-Substitution: Ly = b for y
    y = np.zeros((n,1))
    for i in np.arange(0,n):
        y[i] = b[i].copy()
        for j in np.arange(0,i):
            y[i] = y[i] - L[i,j]*y[j]
            
       
    # Backward-Substitution: Ux = y for x
    x = np.zeros((n,1))
    for i in np.arange(n-1,-1,-1):
        x[i] = y[i].copy()
        for j in np.arange(n-1,i,-1):
            x[i] = x[i] - U[i,j]*x[j]
        x[i] = x[i]/U[i,i]
    
    return np.mat(x)


In [22]:
# Test from HW 1
A = np.matrix([[6,2,-2],[1,4,1],[2,2,5]])
b = np.matrix([[-2],[4],[32]])
LU_Axb_solve(A,b)

matrix([[ 2.],
        [-1.],
        [ 6.]])

### Let's now -- finally -- solve the heat equation!

Run the code below and enjoy the show you created! 

In [23]:
# Number of internal points
N = 50 # Jupyter Hub requires N < 65 for this problem
 
# Calculate Spatial Step-Size
h = 1/(N+1.0)
 
# Create Temporal Step-Size, TFinal, Number of Time-Steps
k = h/2
TFinal = 1
NumOfTimeSteps = int(TFinal/k)
 
# Create grid-points on x axis
x = np.linspace(0,TFinal,N+2)
x = x[1:-1]
 
# Initial Conditions
u = np.transpose(np.mat(100*np.sin(np.pi*x)))

# Calculate Second-Derivative Matrix
data = np.ones((3, N))
data[1] = -2*data[1]
diags = [-1,0,1]
D2 = sparse.spdiags(data,diags,N,N)/(h**2)
D = D2.todense()

# Identity Matrix, A Matrix, b1 Matrix
I = np.identity(N)
A = (I -k/2*D)
b1 = (I + k/2*D) 

# Data for each time-step
data = []
 
for i in range(NumOfTimeSteps):
    # Solve the System: Au = b
    b = b1*u
    u = np.mat(LU_Axb_solve(A,b))
 
    # Save Data
    data.append(u)

In [24]:
# Animate
fig, ax = plt.subplots()

ax.set_xlim((0, TFinal))
ax.set_ylim((1.02*np.min(data), 1.02*np.max(data)))
plt.xlabel("meters along metal rod")
plt.ylabel("temperature")

line, = ax.plot([], [], lw=0.5, c='C1',marker='o')
plt.close(fig)

def init():
    line.set_data([], [])
    return (line,)

def animate(t):
    y = data[t]
    line.set_data(x, y)
    return (line,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20, 
                               blit=True)

HTML(anim.to_jshtml())

&#9989; <font color=red>**DO THIS:**</font>  Before turning in your notebook, verify you have answered all the questions.  
- Use the Red/Bold (<font color=red>**QUESTION**</font> and <font color=red>**DO THIS**</font> )  text as a guild to where you will be evaluated. 
- The check mark symbols (&#9989;) are provided to help you keep track of what you have completed. Feel free to remove the check mark after completing each item. 
- Before submitting your document, Clear and Restart the Kernel and Rerun the entire notebook. **THERE SHOULD BE NO ERRORS**



---------
### Congratulations, you're done with your Homework assignment!


Now, you just need to submit this assignment by uploading it to the course <a href="https://d2l.msu.edu/">Desire2Learn</a> web page for today's dropbox (Don't forget to add your names in the first cell).

&#169; Copyright 2018,  Michigan State University Board of Trustees